**IOC 2: Suspicious Process Spawn via Rundll32.exe**

**1. Attacker Analogy: “The Courier With a Hidden Package”**
Imagine a man walks up to the lobby security desk wearing a city maintenance uniform and driving a municipal water department truck. He looks official. The uniform and vehicle are city-issued—tools that the building trusts. But he’s not there for a real maintenance call. Hidden in the back of the truck is a sealed container that isn’t on any official schedule. He flashes a badge, signs into the log, and heads upstairs without anyone realizing this isn’t a scheduled stop.

In this analogy:

The man is the attacker.

The city-issued truck is rundll32.exe, a legitimate Windows binary.

The uniform and badge represent the inherent trust the system places in signed Microsoft tools.

The unauthorized cargo is the malicious DLL or script.

The building is the Windows OS.

The deception lies not in breaking a window or sneaking in, but in using a trusted vehicle for untrusted delivery. That’s what makes it dangerous.

**2. Source of IOC (Telemetry Origin)**
Telemetry Source:
Windows Security Event Log – Event ID 4688 (New Process Created)
Captured via: EDR (Endpoint Detection and Response) telemetry and native Windows Event Forwarding

**3. Triage Framework Declaration**
Framework:
Host-Based Local Triage Protocol

Standard Investigative Toolkit (in order):

Windows Event Logs

EDR Telemetry (if available)

File System and Registry Inspection

Volatile Memory Capture

Softened Version – What We're Doing Here:
We’re not hunting across the network or examining a phishing link. This is a host-level event, meaning something suspicious occurred on the local system. We’re following our host triage protocol, starting with system logs and moving deeper if needed.

**4. Process or Binary Definition (Required)**
Binary: rundll32.exe
Definition:
rundll32.exe is a legitimate Windows utility that loads and executes DLL (Dynamic Link Library) files and runs specific functions from those DLLs. It is part of the Windows system located in C:\Windows\System32\. While valid, it is often abused by attackers to execute malicious code in a way that blends into normal system activity, making detection harder.

Softened Version – What's This Thing?:
Think of rundll32.exe like a remote control launcher. It lets Windows run small pieces of code hidden inside larger files. Attackers love it because it looks like a system process and can be used to quietly trigger harmful code.

**5. OS Layer with Key Behaviors**
OS Layer:
Layer 1 – Process Execution Environment
Key Behavior:
Abnormal invocation of rundll32.exe launching a suspicious DLL from a non-standard directory (e.g., %AppData% or %Temp%).

Softened Version – What Part of the System's Involved?:
This happens in the process execution layer—where new programs get launched. It’s like watching a new app suddenly open up. But in this case, the way it opened is sketchy, like a random process quietly turning on behind your back.

**6. Cross-Layer Interaction Pivots**
Pivots Observed:

Layer 2 (Startup/Persistence): The same DLL was found registered under a Run key in the registry—suggesting it will auto-launch on reboot.

Layer 3 (Background Execution): A hidden service was also found configured to trigger rundll32 with the same DLL.

Layer 2 – Startup and Persistence Infrastructure
Definition:
This layer covers auto-launch mechanisms that cause a binary, script, or DLL to automatically execute during system boot or user login.

Common mechanisms include:

Registry Run keys:

HKCU\Software\Microsoft\Windows\CurrentVersion\Run

HKLM\Software\Microsoft\Windows\CurrentVersion\Run

Scheduled tasks

Startup folder shortcuts

Login scripts

Service registry keys that specify run-on-boot behavior

In this IOC:

The attacker placed a malicious DLL file path inside a Run key.
This means the DLL will auto-launch using rundll32.exe every time the system reboots.

This is persistence.

This is Layer 2.

Layer 3 – Background Services and Scheduled Execution
Definition:
This layer deals with long-running background processes, typically managed by the Windows Service Control Manager (SCM). These are usually launched at boot but are registered and behave differently than simple Run keys.

Common mechanisms:

Windows services (sc.exe, registry under HKLM\SYSTEM\CurrentControlSet\Services\)

Service binaries

Service startup types (e.g., auto, manual, disabled)

Scheduled task engine (taskschd.msc)

In this IOC:

The attacker registered a new hidden service.

That service was configured to launch rundll32.exe with the same DLL as the argument.

This means the DLL runs in the background, impersonating a legitimate service.

So what's the difference between Layer 2 and Layer 3 in this case?
Feature	Layer 2: Startup & Persistence	Layer 3: Background Execution
Trigger	System boot or user login	Service manager starts it at boot
Mechanism	Run key, scheduled task, startup folder	Registered Windows Service
Execution	One-shot launch (e.g., runs once on login)	Runs persistently in background
Discovery	Registry: Run keys or startup tasks	Services.msc, sc query, registry under Services\
This IOC's Behavior	DLL set to run via Run key using rundll32.exe	Same DLL launched by registered hidden service

Pivot Interpretation in the IOC
Initial indicator: rundll32.exe launched with suspicious DLL (Layer 1)

Pivot to Layer 2: Found DLL in Run key (persistence)

Pivot to Layer 3: Found DLL tied to a hidden Windows Service

Conclusion:
This attacker embedded the same payload at two different layers, making it launch on startup (Layer 2) and run as a service in the background (Layer 3)—a dual persistence mechanism for redundancy.


Softened Version – How It Spreads Inside the System:
After the attacker launched the process, it didn’t stop there. They planted hooks so the same thing would run again next time the computer restarts (Layer 2), and even set up a quiet background process to do it regularly (Layer 3). It’s like setting up a machine to keep calling back every day.

**7. OSI Layer Relevance**
Primary OSI Layer:

Layer 7 – Application (abnormal application-layer invocation via a Windows utility)

Secondary (Potential Pivot):

Layer 4 – Transport (if the payload connects out, future steps may pivot to examining TCP connections)

Any observable invocation of a Windows utility (like rundll32.exe) that accepts input parameters and executes in user space is an OSI Layer 7 behavior because it represents application-level interaction with the OS, whether for legitimate or malicious purposes.

Softened Version – How Deep in the Stack Is This?:
This happened high up—in the application layer—where user-facing software and utilities live. But if this malware starts talking to a remote server, we’d have to look at the transport layer too. Right now, though, it's just local trickery.

**8. Attacker Behavior Interpretation**
Interpretation:
The attacker intentionally used rundll32.exe to execute a malicious DLL in a stealthy way, avoiding launching a full executable (EXE) which might raise red flags. By nesting the DLL in a user-writable directory, they bypassed traditional application whitelisting. Registry-based persistence and service creation indicate an intent to maintain access over time.

Softened Version – What the Attacker's Trying to Do:
The attacker is trying to sneak in quietly—not with a big obvious program, but with a clever disguise. They use a normal Windows tool to launch their code, hide it in a familiar location, and make sure it comes back even after a reboot.

**9. Defender Action Summary**
Flag and alert on rundll32.exe executing from non-standard paths (e.g., %AppData%, %Temp%).

Investigate parent-child process relationships (e.g., what spawned rundll32?).

Inspect registry Run keys for unauthorized DLL entries.

Quarantine the malicious DLL.

Review persistence mechanisms using tools like Autoruns.

Enable EDR detection rules for known rundll32 abuse patterns.

Softened Version – What Should We Do About It?
We should treat any use of rundll32 that doesn’t come from its normal location as suspicious. Track where it came from, remove the hidden DLL, and check the system’s “autopilot settings” (like the registry) to make sure it doesn’t come back. Use EDR tools to catch this pattern in the future.

Note: In this scenario, memory forensics was not conducted because the process had terminated and disk artifacts were recovered. However, if rundll32.exe had still been running, volatile memory capture would have been essential to fully inspect the injected DLL’s behavior.



**10. Attacker Strategy Notes**
The attacker deliberately leveraged a trusted system utility (rundll32.exe) to gain execution in a manner that avoids traditional signature-based detection. Their goal was stealth and persistence—executing their payload without writing an obvious EXE file, and ensuring it restarts automatically. This technique is often seen in fileless malware strategies and living off the land (LOTL) attacks, where the attacker uses legitimate tools against the system. The use of both registry persistence and service installation suggests this was not opportunistic, but part of a planned intrusion.


**Summary**

In this second case study of the structured IOC triage series, we examined a subtle but dangerous host-based compromise involving the abuse of the Windows utility `rundll32.exe` to execute a malicious DLL payload. Unlike more overt attacks, this technique blended into legitimate system behavior, requiring layered investigation across Windows Event Logs, EDR telemetry, registry keys, and file system artifacts.

**Attacker Narrative**

The adversary in this case didn’t kick down the door or pick the lock—they walked in through the front entrance, wearing a city uniform and driving a sanitation truck. They exploited a system-native utility (`rundll32.exe`) to deliver a weaponized DLL in plain sight, bypassing suspicion and avoiding noisy payload droppers or shellcode.

What made this especially deceptive was the attacker’s use of persistence infrastructure: the DLL was configured to auto-launch via a `Run` registry key and was also embedded into a background service. These redundant execution paths suggest an attacker seeking both reliability and stealth.

**Investigation Highlights**

- **Windows Security Event Log (4688)** captured the initial `rundll32.exe` process creation with suspicious command-line parameters.
- **EDR telemetry** corroborated the anomalous execution path and flagged the process for behavioral deviations.
- **Registry analysis** revealed persistence via the `HKCU\Software\Microsoft\Windows\CurrentVersion\Run` key.
- **A background service** was registered to trigger the same DLL, indicating a second persistence channel.

**Cross-Layer Insight**

This case demonstrated clear pivot points between:
- **Layer 1** (Process Execution)
- **Layer 2** (Startup and Persistence)
- **Layer 3** (Background Service Infrastructure)

The attack's execution strategy moved fluidly across these layers, reinforcing the need for defenders to analyze indicators not in isolation, but as behavioral threads woven through OS subsystems.

**Analyst Reflection**

While the forensic team successfully captured disk-based indicators, **volatile memory capture was not exercised**. In this case, disk-based indicators included the suspicious DLL file itself, the rundll32.exe process creation event (Event ID 4688), a persistent Run key in the Windows registry, and a registered Windows service configured to invoke the same DLL—each leaving concrete forensic artifacts on disk.


While the forensic team successfully captured disk-based indicators—including the malicious DLL, a process creation event (4688), a persistent Run key, and a Windows service—all of which were recovered through post-mortem analysis, volatile memory capture was not exercised. Because this case involved direct DLL execution (not injection), disk-based forensics were sufficient. However, in future investigations involving stealthier execution techniques like DLL injection or process hollowing, live memory analysis will be essential for capturing in-memory artifacts and runtime behavior.

This case continues the adversary-first, structured triage model defined in the Engineered Cybersecurity Architecture. It reflects real-world analyst workflows and emphasizes investigative rigor, system-layer awareness, and attacker behavior over static IOC lists.

